In [0]:
!pip install -U catboost
!pip install  eli5
!pip install  shap
!pip install -U ipykernel

Requirement already up-to-date: catboost in /usr/local/lib/python3.6/dist-packages (0.21)
Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.1.4)


In [0]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline


import gc
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
import warnings 
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
import warnings 
from lightgbm import LGBMClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.tree import DecisionTreeClassifier
dot_data = StringIO()
warnings.filterwarnings('ignore')


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
path = '/content/gdrive/My Drive/Zeta Team/'

In [0]:
#reding identity and transaction file of both train and test
identity_train = pd.read_csv(path+'train_identity.csv')
identity_test = pd.read_csv(path+'test_identity.csv')
test_transaction = pd.read_csv(path+'test_transaction.csv')
train_transaction = pd.read_csv(path+'train_transaction.csv')
sample_sub=pd.read_csv(path+'sample_submission.csv')


In [0]:
#Merging both the files on the basis of TransactionID column
train = pd.merge(train_transaction,identity_train, on='TransactionID', how='left',left_index=True,right_index=True)
test = pd.merge(test_transaction,identity_test, on='TransactionID', how='left',left_index=True,right_index=True)

In [0]:
#lets first delete the individual files since we merge them into one
del train_transaction,test_transaction,identity_train,identity_test


In [0]:
train0=train.copy()
test0=test.copy()

In [0]:
def data_drop(data,percent):
  list_drop=[]
  for i in data.columns:
    per=np.round(data[i].isnull().sum()/data.shape[0]*100,2)
    if per>=percent :
      list_drop.append(i)
  return list_drop

In [0]:
vfeatures = []
for i in train.columns:
    if 'V' in i:
        vfeatures.append(i)
len(vfeatures)

339

In [0]:
cfeatures = []
for i in train.columns:
    if 'C' in i:
        cfeatures.append(i)
len(cfeatures)

15

In [0]:
idfeatures = []
for i in test.columns:
    if 'id' in i:
        idfeatures.append(i)
len(idfeatures)

38

In [0]:
train.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

In [0]:
train.drop(['TransactionID','ProductCD','card4','card6','P_emaildomain','R_emaildomain','id_01','id_02','id_03','id_04','id_05','id_06'
           ,'id_07','id_08','id_09','id_10','id_11','id_12','id_13','id_14','id_15','id_16'
           ,'id_17','id_18','id_19','id_20','id_21','id_22','id_23','id_24','id_25','id_26'
           ,'id_27','id_28','id_29','id_30','id_31','id_32','id_33','id_34','id_35','id_36','id_37','id_38','DeviceInfo','DeviceType'],axis=1,inplace=True)

In [0]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3',
       'card5', 'addr1', 'addr2', 'dist1',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=388)

In [0]:
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.949999999999999,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,...,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663550,18403263,49.000000000000000,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,...,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,Found,NaN,-300.0,Found,Found,166.0,NaN,542.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Android 6.0.1,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663551,18403310,171.000000000000000,W,4476,574.0,150.0,visa,226.0,debit,472.0,87.0,2635.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,NaN,136.0,...,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,-360.0,New,NotFound,225.0,NaN,271.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663552,18403310,284.949999999999989,W,10989,360.0,150.0,visa,166.0,debit,205.0,87.0,17.0,NaN,gmail.com,NaN,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,NaN,242.0,...,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663553,18403317,67.950000000000003,W,18018,452.0,150.0,mastercard,117.0,debit,264.0,87.0,6.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,22.0,...,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,567.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [0]:
test.drop(['TransactionID','ProductCD','card4','card6','P_emaildomain','R_emaildomain','DeviceInfo','DeviceType'],axis=1,inplace=True)

In [0]:
test.drop(idfeatures,axis=1,inplace=True)

In [0]:
le=LabelEncoder()

In [0]:
def imputation(data):
   for i in data.columns:
      if data[i].dtypes=='int64' or data[i].dtypes=='float64':
         data[i].fillna(data[i].mean(),inplace=True)
      if data[i].dtypes=='object':
         data[i].fillna(data[i].mode()[0],inplace=True)
      if 'M' in i:
         data[i].fillna(data[i].mode()[0],inplace=True)
         data[i]=le.fit_transform(data[i])
   return data

In [0]:
train=imputation(train)
test=imputation(test)

In [0]:
test.head()

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,18403224,31.949999999999999,10409,111.0,150.0,226.0,170.0,87.0,1.0,237.175046657152279,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,82.443145300241213,61.815641993580897,160.834483184871516,0.55398057899222,418.0,203.0,77.404178819995963,18.225961226739287,58.163185582582429,409.0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000000000000,47.950000762939453,0.000000000000000,0.000000000000000,47.950000762939453,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.436681451243605,1.216181915714828,0.693514411691591,0.047925579934843,0.482657560413968,0.163271447449333,0.220486451832372,0.460572280880794,0.309900433070351,315.12861322419252,440.464332963029335,381.102128692689632,24.708030118030273,58.339467875821576,35.485643387464698,99.058759682119017,155.578152729436738,139.814706222135499
1,18403263,49.000000000000000,4272,111.0,150.0,226.0,299.0,87.0,4.0,237.175046657152279,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,82.443145300241213,61.815641993580897,160.834483184871516,0.55398057899222,231.0,634.0,77.404178819995963,18.225961226739287,58.163185582582429,634.0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000000000000,280.000000000000000,77.000000000000000,0.000000000000000,280.000000000000000,0.000000000000000,77.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.436681451243605,1.216181915714828,0.693514411691591,0.047925579934843,0.482657560413968,0.163271447449333,0.220486451832372,0.460572280880794,0.309900433070351,315.12861322419252,440.464332963029335,381.102128692689632,24.708030118030273,58.339467875821576,35.485643387464698,99.058759682119017,155.578152729436738,139.814706222135499
2,18403310,171.000000000000000,4476,574.0,150.0,226.0,472.0,87.0,2635.0,237.175046657152279,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,82.443145300241213,61.815641993580897,160.834483184871516,0.55398057899222,136.0,136.0,77.404178819995963,18.225961226739287,58.163185582582429,97.0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000000000000,1321.000000000000000,0.000000000000000,0.000000000000000,1058.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.0,0.0,0.0,0.0,263.0,0.0,0.436681451243605,1.216181915714828,0.693514411691591,0.047925579934843,0.482657560413968,0.163271447449333,0.220486451832372,0.460572280880794,0.309900433070351,315.12861322419252,440.464332963029335,381.102128692689632,24.708030118030273,58.339467875821576,35.485643387464698,99.058759682119017,155.578152729436738,139.814706222135499
3,18403310,284.949999999999989,10989,360.0,150.0,166.0,205.0,87.0,17.0,237.175046657152279,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,82.443145300241213,61.815641993580897,160.834483184871516,0.55398057899222,242.0,242.0,77.404178819995963,18.225961226739287,58.163185582582429,242.0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,282.540008544921875,282.540008544921875,282.540008544921875,0.0,0.0,0.0,0.0,0.0,0.0,0.436681451243605,1.216181915714828,0.693514411691591,0.047925579934843,0.482657560413968,0.163271447449333,0.220486451832372,0.460572280880794,0.309900433070351,315.12861322419252,440.464332963029335,381.102128692689632,24.708030118030273,58.339467875821576,35.485643387464698,99.058759682119017,155.578152729436738,139.814706222135499
4,18403317,67.950

In [0]:
train.head()

,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,86400,68.5,13926,362.555488499966486,150.0,142.0,315.0,87.0,19.000000000000000,231.855422967549885,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,169.563231453172477,13.000000000000000,140.002440597887187,42.335964859155183,69.805716862284285,41.638949559318547,146.05810764687871,0.561057126596166,13.000000000000000,13.00000000000000,54.037532642118762,17.901294550619834,57.724444015630276,0.000000000000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,6.220288763949436,13.103775303275006,9.184612208716349,0.05849352163301,0.851040060229991,0.296632706342364,0.336790081480492,1.312843802746779,0.77587400274435,721.741882916404506,1375.783644036513351,1014.62278150914301,9.807015093927214,59.164549671486512,28.530902964203371,55.35242195053889,151.160542070513145,100.700881592801892
1,0,86401,29.0,2755,404.000000000000000,150.0,102.0,325.0,87.0,118.502180308810637,231.855422967549885,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,169.563231453172477,28.343347718075332,0.000000000000000,42.335964859155183,69.805716862284285,41.638949559318547,146.05810764687871,0.561057126596166,0.000000000000000,146.62146549591489,54.037532642118762,17.901294550619834,57.724444015630276,0.000000000000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220288763949436,13.103775303275006,9.184612208716349,0.05849352163301,0.851040060229991,0.296632706342364,0.336790081480492,1.312843802746779,0.77587400274435,721.741882916404506,1375.783644036513351,1014.62278150914301,9.807015093927214,59.164549671486512,28.530902964203371,55.35242195053889,151.160542070513145,100.700881592801892
2,0,86469,59.0,4663,490.000000000000000,150.0,166.0,330.0,87.0,287.000000000000000,231.855422967549885,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,169.563231453172477,28.343347718075332,0.000000000000000,42.335964859155183,69.805716862284285,41.638949559318547,146.05810764687871,0.561057126596166,0.000000000000000,315.00000000000000,54.037532642118762,17.901294550619834,57.724444015630276,315.000000000000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.220288763949436,13.103775303275006,9.184612208716349,0.05849352163301,0.851040060229991,0.296632706342364,0.336790081480492,1.312843802746779,0.77587400274435,721.741882916404506,1375.783644036513351,1014.62278150914301,9.807015093927214,59.164549671486512,28.530902964203371,55.35242195053889,151.160542070513145,100.700881592801892
3,0,86499,50.0,18132,567.000000000000000,150.0,117.0,476.0,87.0,118.502180308810637,231.855422967549885,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.000000000000000,0.000000000000000,94.000000000000000,0.000000000000000,69.805716862284285,41.638949559318547,146.05810764687871,0.561057126596166,84.000000000000000,146.62146549591489,54.037532642118762,17.901294550619834,57.724444015630276,111.000000000000000,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,6.220288763949436,13.103775303275006,9.184612208716349,0.05849352163301,0.851040060229991,0.296632706342364,0.336790081480492,1.312843802746779,0.77587400274435,721.741882916404506,1375.783644036513351,1014.62278150914301,9.807015093927214,59.164549671486512,28.530902964203371,55.35242195053889,151.160542070513145,100.700881592801892
4,0,86506,50.0,4497,514.000000000000000,150.0,102.0,420.0,87.0,118.502180308810637,231.855422967549885,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,16

In [0]:
def model_results(model,X,test,title='naif_model'):
  n_cv=int(X.shape[0]*0.7)
  train = X[:n_cv]
  cv = X[n_cv:]
  train_x = train.drop(['isFraud'],axis=1)
  train_y = train['isFraud']
  cv_x = cv.drop(['isFraud'],axis=1)
  cv_y = cv['isFraud']
  model.fit(train_x,train_y)
  predict_y_=rf.predict_proba(train_x)
  predict_y = rf.predict_proba(cv_x)
  print('train auc:',roc_auc_score(train_y,predict_y_[:,1]))
  print('cv auc:',roc_auc_score(cv_y,predict_y[:,1]))
  predict=model.predict_proba(test)
  sample_sub['isFraud']=predict[:,1]
  sample_sub.to_csv(path+title+'.csv',index=False)




SUb 3 

---



In [0]:
def get_data():
  train_x = train.drop(['isFraud'],axis=1)
  train_y = train['isFraud']
  return train_x,train_y,test
  
train_x,train_y,test=get_data()


In [0]:
train_x.head()

In [0]:
n_fold = 5
folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }


In [0]:
result_dict_lgb = train_model_classification(X=v_train_x, X_test=test, y=v_train_y, params=params, folds=folds, model_type='lgb', eval_metric='auc', plot_feature_importance=True,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=5000, averaging='usual', n_jobs=-1)

In [0]:
sub=sample_sub.copy()
sub['isFraud'] = result_dict_lgb['prediction']
sub.to_csv(path+'submission3.csv', index=False)

In [0]:
train.head()

SUB 4 : 

LSTM 

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Embedding
from keras.layers.recurrent import LSTM


In [0]:
train_x.shape,test.shape

((590540, 387), (506691, 387))

In [0]:
embed_dim = 556
lstm_out = 200
batch_size = 524
epochs=5

model = Sequential()
model.add(Embedding(train_x.shape[1], embed_dim, dropout = 0.1))
model.add(LSTM(lstm_out, dropout_U = 0.1, dropout_W = 0.1))
model.add(Dense(256, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adadelta',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 556)         215172    
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               605600    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               51456     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 872,485
Trainable params: 872,485
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(train_x, train_y, validation_split=0.33, epochs=epochs, batch_size=batch_size)


In [0]:
model.save('lstm_model2.h5')


balance data

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
class_weights

array([ 0.51812935, 14.28979335])

In [0]:
model.fit(train_x, train_y, validation_split=0.33, epochs=epochs, batch_size=batch_size,class_weight=class_weights,shuffle=True)

Train on 395661 samples, validate on 194879 samples
Epoch 1/5
395661/395661 [==============================] - 728s 2ms/step - loss: 0.1544 - acc: 0.9642 - val_loss: 0.1575 - val_acc: 0.9641
Epoch 2/5
395661/395661 [==============================] - 702s 2ms/step - loss: 0.1467 - acc: 0.9655 - val_loss: 0.1601 - val_acc: 0.9641
Epoch 3/5
395661/395661 [==============================] - 699s 2ms/step - loss: 0.1372 - acc: 0.9655 - val_loss: 0.1430 - val_acc: 0.9641
Epoch 4/5
395661/395661 [==============================] - 706s 2ms/step - loss: 0.1364 - acc: 0.9655 - val_loss: 0.1482 - val_acc: 0.9641
Epoch 5/5
395661/395661 [==============================] - 691s 2ms/step - loss: 0.1414 - acc: 0.9655 - val_loss: 0.1458 - val_acc: 0.9641


In [0]:
model.save('lstm_model_umbalencednew.h5')

In [0]:
# load and evaluate a saved model
from keras.models import load_model

# load model
model = load_model('lstm_model_umbalenced.h5')
# summarize model.
model.summary()

In [0]:
test_predict = model.predict(test)

In [0]:
sub=sample_sub.copy()
sub['isFraud'] = test_predict
sub.to_csv(path+'submission_lstm_umbalenced.csv', index=False)